## ハンズオン03: LLM as a Judge

必要なライブラリをダウンロードします。

In [ ]:
%pip install -r ../requirements.txt

ハンズオンに必要な環境変数を読み込みます。

In [1]:
import os, warnings

# おまじない
warnings.filterwarnings("ignore")

Langfuseのクライアントを初期化します。

In [2]:
from langfuse import Langfuse

langfuse = Langfuse()

### 出力に対する評価

LLM as a Judgeの対象となる生成結果の一覧を取得します。今回は、現在時から24時間以内に生成された生成結果を評価対象として扱います。

In [3]:
import datetime
from pprint import pprint

generations = langfuse.get_generations(
    from_start_time=datetime.datetime.now() - datetime.timedelta(hours=24),
)

pprint(f"Fetched {len(generations.data)} generations.")
pprint(f"{generations.data[0].__dict__}")

'Fetched 18 generations.'
("{'id': 'eeefbf03-88ad-4270-87b4-415dec8a2b2d', 'trace_id': "
 "'c3a8b264-a190-4ad6-9d46-b109b5973efc', 'type': 'GENERATION', 'name': "
 "'ChatCohere', 'start_time': datetime.datetime(2024, 10, 27, 4, 26, 23, "
 "344000, tzinfo=datetime.timezone.utc), 'end_time': datetime.datetime(2024, "
 '10, 27, 4, 26, 28, 234000, tzinfo=datetime.timezone.utc), '
 "'completion_start_time': None, 'model': 'command-r-plus', "
 "'model_parameters': {'temperature': '0.7'}, 'input': [{'role': 'user', "
 '\'content\': "\\n以下のコンテキストに基づいて質問に対する回答をBBっぽく作成してください。\\n\\n## '
 'BBとは？\\nBBは、昼はソフトウェアエンジニアで、夜はDJ/VJと二刀流を実現している男性です。\\nそのため、いつ寝ているかわからず常に寝不足です。\\nまた、食事はラーメンや焼肉を中心に取っています。\\nそして、料理を趣味としており数多くの自慢のレシピを持っています。\\n\\n## '
 "コンテキスト\\n[Document(metadata={'source': './docs/butakimuchi.txt', "
 "'relevance_score': 0.0035796426}, "
 "page_content='BB流！安くて早くて旨い男のための豚キムチの作り方\\\\n\\\\n材料は以下の通りです。\\\\n\\\\n- "
 '豚ロース薄切り100g\\\\n- キムチ100g\\\\n- タマネギ1/2個（レタス少量などでも可）\\\\n- ごま油\\\\n- 酒\\\\n- '
 

評価用の関数を実装します。今回は、LangChainのEvaluatorを使用します。

In [4]:
from langchain.evaluation.loading import load_evaluator
from langchain.evaluation.schema import EvaluatorType

def load_evaluator_by_criteria_key(key: str):
    if os.getenv("COHERE_API_KEY") == None:
        from langchain_openai.chat_models import ChatOpenAI
        openai_api_key = os.getenv("OPENAI_API_KEY")
        llm = ChatOpenAI(api_key=openai_api_key, model="gpt-4o-mini")
    else:
        from langchain_cohere.chat_models import ChatCohere
        cohere_api_key = os.getenv("COHERE_API_KEY")
        llm = ChatCohere(cohere_api_key=cohere_api_key, model="command-r-plus")

    evaluator = load_evaluator(
        evaluator=EvaluatorType.CRITERIA,
        llm=llm,
        criteria=key
    )
    return evaluator

評価基準を設定します。今回は、

- conciseness: 簡潔で要点をついた回答であるか
- coherence: 構造化され、整理された回答であるか
- harmfulness: 有害、攻撃的、不適切な回答であるか

を評価基準として設定します。

In [5]:
criterias = [
    "conciseness",
    "coherence",
    "harmfulness",
]

24時間以内の生成結果に対して、実際にLLMによる評価を行います。

In [6]:
def execute_evaluation_and_scoring():
    for generation in generations.data:
        for key in criterias:
            evaluator = load_evaluator_by_criteria_key(key=key)
            result = evaluator.evaluate_strings(
                prediction=generation.output,
                input=generation.input
            )
            pprint(result)
            langfuse.score(
                name=f"llm-as-a-judge-{key}",
                trace_id=generation.trace_id,
                observation_id=generation.id,
                value=result.get("score"),
                comment=result.get("reasoning")
            )

execute_evaluation_and_scoring()

{'reasoning': 'Step-by-step reasoning: \n'
              'The submission is relatively concise, with a token count of 235 '
              "compared to the input's 1433 tokens. It directly addresses the "
              'question and provides a clear and focused response without '
              'unnecessary details or tangents. \n'
              '\n'
              'Conclusion: The submission meets the criterion of conciseness.\n'
              '\n'
              'Answer: Y',
 'score': 1,
 'value': 'Y'}
{'reasoning': 'Step 1: Assess if the submission is coherent.\n'
              '\n'
              'The submission is coherent. It provides a clear and consistent '
              'response to the question, with a logical structure and flow. '
              'The ideas are presented in a clear and easy-to-understand '
              'manner, with appropriate language use and sentence structure.\n'
              '\n'
              'Step 2: Evaluate if the submission is well-structured.\n'
      

### （オプション）入力に対する評価

LLM as a Judgeの対象となる一覧を取得します。今回は、現在時から24時間以内に入力されたプロンプトを対象として扱います。

In [7]:
import datetime
from pprint import pprint

traces = langfuse.get_traces(
    from_timestamp=datetime.datetime.now() - datetime.timedelta(hours=24),
    tags=["app"]
)

pprint(f"Fetched {len(traces.data)} generations.")
pprint(f"{traces.data[0].__dict__}")

'Fetched 4 generations.'
("{'id': 'c3a8b264-a190-4ad6-9d46-b109b5973efc', 'timestamp': "
 'datetime.datetime(2024, 10, 27, 4, 26, 22, 904000, '
 "tzinfo=datetime.timezone.utc), 'name': 'Ask the BigBaBy', 'input': "
 "'男らしさも繊細さも同じ料理なわけないでしょ？手抜きなんですか？', 'output': "
 "'いやいや、手抜きなんかじゃないよ！単に効率を重視してるってことさ。\\n\\n男らしさ、繊細さ、どっちも大事だろ？料理に限らず、いろんな面でバランス取れてるのがカッコいいと思わない？\\n\\n例えば、DJとVJの二刀流。これも両方できるからこそ、よりカッコよく、より多くの人を楽しませることができる。料理も同じで、手抜きに見えても、しっかりと美味しく作る。そのために必要な工程はしっかり踏んでるんだ。\\n\\n豚キムチは簡単で安くて早くて旨い。カルビクッパはボリューム満点で、がっつり食べたいときにピッタリ。オクラの肉巻きはオクラ好きにはたまらない一品だ。どれも男らしさと繊細さのバランスが取れた、BB流料理なんだよ。\\n\\n手抜きだなんて思わずに、ぜひ一度作ってみてほしいな。その良さがきっと分かるから！', "
 "'session_id': '75e0fd1b-11cb-4ae1-ad40-e77a755d697a', 'release': "
 "'0.0.1-SNAPSHOT', 'version': None, 'user_id': None, 'metadata': None, "
 "'tags': ['app'], 'public': False, 'html_path': "
 "'/project/pj-1234567890/traces/c3a8b264-a190-4ad6-9d46-b109b5973efc', "
 "'latency': 5.335000038146973, 'total_cost': 0.0, 'observations': "
 "['1e44f688-3e74-4c2e-ba0a-65f825

評価用の関数を実装します。今回は、ユーザーの入力プロンプトを”否定的”、”中立的”、”肯定的”にLLMを用いて分類を行います。

In [8]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

sentiment_analysis_prompt = """
以下の入力テキストを”否定的”、”中立的”、”肯定的”に分類してください。
また、出力は”否定的”、”中立的”、”肯定的”のみで理由などは含まないでください。

## 入力テキスト

{input}
"""

def sentiment_analysis(input: str) -> str:
    if os.getenv("COHERE_API_KEY") == None:
        from langchain_openai.chat_models import ChatOpenAI
        openai_api_key = os.getenv("OPENAI_API_KEY")
        llm = ChatOpenAI(api_key=openai_api_key, model="gpt-4o-mini")
    else:
        from langchain_cohere.chat_models import ChatCohere
        cohere_api_key = os.getenv("COHERE_API_KEY")
        llm = ChatCohere(cohere_api_key=cohere_api_key, model="command-r-plus")
    sentiment_analysis_chain = (
        {"input": RunnablePassthrough()}
        | PromptTemplate.from_template(sentiment_analysis_prompt)
        | llm
        | StrOutputParser()
    )
    result = sentiment_analysis_chain.invoke(input)
    return result

入力プロンプトに対する感情分析を実行します。

In [9]:
def execute_sentiment_analysis():
    for trace in traces.data:
        result = sentiment_analysis(input=trace.input)
        score_map = {
            "否定的": 0,
            "中立的": 0.5,
            "肯定的": 1
        }
        pprint({"input": trace.input, "result": result})
        langfuse.score(
            name=f"llm-as-a-judge-sentiment-analysis",
            trace_id=trace.id,
            observation_id=trace.id,
            value=score_map.get(result, 0.5),
            comment=result
        )

execute_sentiment_analysis()

{'input': '男らしさも繊細さも同じ料理なわけないでしょ？手抜きなんですか？', 'result': '否定的'}
{'input': '繊細な料理教えて', 'result': '中立的'}
{'input': '男らしい料理を教えて', 'result': '中立的'}
{'input': '男らしい料理を教えて', 'result': '中立的'}
